In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, d_model = 2, row_dim = 0, col_dim=1):
        # d_model = dimensi dari token encoding
        # row_dim = dimensi baris dari matriks
        # col_dim = dimensi kolom dari matriks

        super().__init__()
        ## inisialisasi Weight(W) dan Bias(b) untuk Query, Key, Value
        # untuk setiap token/kata
        ## in_feature = baris ; out_features = kolom
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False) # W untuk Query
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False) # W untuk Key
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False) # W untuk Value
        # nn.Linear tidak hanya membuat matriks, tapi juga melakukan operasi math

        self.row_dim = row_dim
        self.col_dim = col_dim

    # operasi forward untuk menghitung self-attention score untuk setiap token
    def forward(self, token_embeddings):
        ## membuat Query, Key, dan Value dari nilai encoding
        # yang berasal dari setiap token (token_encodings)
        ## Encoded values matmul with their Weight and store in Q/K/V
        q = self.W_q(token_embeddings)
        k = self.W_k(token_embeddings)
        v = self.W_v(token_embeddings)

        ## calcute attention-score ## 

        # similiraity score : (q * k^T) -> numerator
        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        # normalisasi similarity score //  divide by (sqrt(d_k)) -> denominator
        # d_k = dimensi dari key
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        # softmax
        attention_sm = F.softmax(scaled_sims, dim=self.col_dim)

        # kalikan dengan Value (v)
        attention_score = torch.matmul(attention_sm, v)

        ## end ##

        return attention_score